In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
import numpy as np
import os,cv2,io
import sklearn

TensorFlow 2.x selected.
2.1.0-rc1


In [0]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  

In [0]:
data_path = '/content/drive/My Drive/DeepFakeFiles/train.tfrecords'
file_name = '/content/drive/My Drive/DeepFakeFiles/val.tfrecords'

In [0]:
import cv2
filenames = [data_path]
raw_dataset = tf.data.TFRecordDataset(filenames)
val_dataset = tf.data.TFRecordDataset([file_name]) 

feature = {'train/image': tf.io.FixedLenFeature([], tf.string),
            'train/label': tf.io.FixedLenFeature([], tf.int64),
           'train/audio': tf.io.FixedLenFeature([], tf.string),}


features_val = {'val/image': tf.io.FixedLenFeature([], tf.string),
            'val/label': tf.io.FixedLenFeature([], tf.int64),
           'val/audio': tf.io.FixedLenFeature([], tf.string),}

def _parse_function_val(example_proto):
  features = tf.io.parse_single_example(example_proto, features_val)
  audio_raw = tf.io.decode_raw(features['val/audio'],tf.float32)
  audio = tf.reshape(audio_raw,shape=(105000,1))

  image_raw = tf.io.decode_raw(features['val/image'],tf.float32)
  image = tf.reshape(image_raw,[5,500,500,3])
  image = tf.image.rgb_to_grayscale(image)
  image = tf.image.resize(image,(350,350))
  labels =features['val/label']
  labels = tf.one_hot(labels,depth=1) 
  return (image,audio),labels


def _parse_function(example_proto):
  features = tf.io.parse_single_example(example_proto, feature)
  audio_raw = tf.io.decode_raw(features['train/audio'],tf.float32)
  audio = tf.reshape(audio_raw,shape=(105000,1))

  image_raw = tf.io.decode_raw(features['train/image'],tf.float32)
  image = tf.reshape(image_raw,[5,500,500,3])
  image = tf.image.rgb_to_grayscale(image)
  image = tf.image.resize(image,(350,350))

  labels =features['train/label']
  labels = tf.one_hot(labels,depth=1)
  return (image,audio),labels

dataset = raw_dataset.map(_parse_function)
print(dataset)
val = val_dataset.map(_parse_function_val)
print(val)

<MapDataset shapes: (((5, 350, 350, 1), (105000, 1)), (1,)), types: ((tf.float32, tf.float32), tf.float32)>
<MapDataset shapes: (((5, 350, 350, 1), (105000, 1)), (1,)), types: ((tf.float32, tf.float32), tf.float32)>


In [0]:
BATCH_SIZE = 4
BUFFER_SIZE = 10

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
print(dataset)
valid = val.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
valid

<RepeatDataset shapes: (((None, 5, 350, 350, 1), (None, 105000, 1)), (None, 1)), types: ((tf.float32, tf.float32), tf.float32)>


<RepeatDataset shapes: (((None, 5, 350, 350, 1), (None, 105000, 1)), (None, 1)), types: ((tf.float32, tf.float32), tf.float32)>

In [0]:
def action_model(shape):
    inputs = tf.keras.layers.Input(shape=shape)
    convLSTM1 = tf.keras.layers.ConvLSTM2D(filters=16,kernel_size=(3,3),return_sequences=True)(inputs)
    BN1 = tf.keras.layers.BatchNormalization()(convLSTM1)
    # convLSTM2 = tf.keras.layers.ConvLSTM2D(filters=32,kernel_size=(2,2),return_sequences=True)(BN1)
    # BN2= tf.keras.layers.BatchNormalization()(convLSTM2)
    flatten = tf.keras.layers.Flatten()(BN1)
    # dense1 = tf.keras.layers.Dense(64)(flatten)
    dense2 = tf.keras.layers.Dense(32,activation='relu')(flatten)
    
    model = tf.keras.models.Model(inputs=inputs,outputs=dense2)
    print(model.summary())
    return model

In [0]:
def audio_and_final_model(shape):
  input_shape = tf.keras.layers.Input(shape=shape)
  Conv1 = tf.keras.layers.Conv1D(16,activation='relu',kernel_size=(10))(input_shape)
  MaxPool1 = tf.keras.layers.MaxPool1D()(Conv1)
  BatchNorm1 = tf.keras.layers.BatchNormalization()(MaxPool1)
  Dropout1 = tf.keras.layers.Dropout(0.4)(BatchNorm1)
  Conv2 = tf.keras.layers.Conv1D(32,activation='relu',kernel_size=(10))(Dropout1)
  MaxPool2 = tf.keras.layers.MaxPool1D()(Conv2)
  # Dropout2 = tf.keras.layers.Dropout(0.4)(MaxPool2)
  # Conv3 = tf.keras.layers.Conv1D(16,activation='relu',kernel_size=(10))(Dropout1)
  # MaxPool3 = tf.keras.layers.MaxPool1D()(Conv3)
  # Dropout3 = tf.keras.layers.Dropout(0.4)(MaxPool3)
  Flatten = tf.keras.layers.Flatten()(MaxPool2)
  # Dense1 = tf.keras.layers.Dense(128,activation='relu')(Flatten)
  # Dense2 = tf.keras.layers.Dense(64,activation='relu')(Flatten)
  Dense3 = tf.keras.layers.Dense(32,activation='relu')(Flatten)

  model = tf.keras.models.Model(inputs=input_shape,outputs=Dense3)
  
  return model

In [0]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [0]:
am = action_model((5, 350, 350, 1))
afm = audio_and_final_model((105000,1))

combined = tf.keras.layers.Concatenate(axis=-1)([am.output,afm.output])
z = tf.keras.layers.Dense(32,activation='relu')(combined)
# den1 = tf.keras.layers.Dense(64,activation='relu')(z)
# den2 = tf.keras.layers.Dense(32,activation='relu')(z)
dropOut = tf.keras.layers.Dropout(0.3)(z)
out = tf.keras.layers.Dense(1,activation='sigmoid')(dropOut)
model = tf.keras.models.Model(inputs=[am.input,afm.input],outputs=out)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5, 350, 350, 1)]  0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 5, 348, 348, 16)   9856      
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 348, 348, 16)   64        
_________________________________________________________________
flatten (Flatten)            (None, 9688320)           0         
_________________________________________________________________
dense (Dense)                (None, 32)                310026272 
Total params: 310,036,192
Trainable params: 310,036,160
Non-trainable params: 32
_________________________________________________________________
None
Model: "model_2"
__________________________________________________________________________________________

In [0]:
EPOCHS=100
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [tf.keras.callbacks.ReduceLROnPlateau(verbose=1,monitor='loss',patience=5,min_delta=1e-6)]

print("Starting training")
model.fit(dataset,epochs=EPOCHS,callbacks=callbacks,steps_per_epoch=80,validation_data=valid,validation_steps=20)
print("Saving Model")
model.save("/content/drive/My Drive/DeepFakeFiles/DeepFakeModel.h5")

Starting training
Train for 80 steps, validate for 20 steps
Epoch 1/100
80/80 [==============================] - 127s 2s/step - loss: 287.3501 - accuracy: 0.6375 - val_loss: 5.9388 - val_accuracy: 0.5250
Epoch 2/100
80/80 [==============================] - 44s 546ms/step - loss: 17.3463 - accuracy: 0.7031 - val_loss: 0.5768 - val_accuracy: 0.8250
Epoch 3/100
80/80 [==============================] - 44s 547ms/step - loss: 0.6280 - accuracy: 0.7906 - val_loss: 0.4541 - val_accuracy: 0.8250
Epoch 4/100
80/80 [==============================] - 44s 545ms/step - loss: 0.5425 - accuracy: 0.8000 - val_loss: 0.4423 - val_accuracy: 0.8250
Epoch 5/100
80/80 [==============================] - 44s 545ms/step - loss: 0.4927 - accuracy: 0.7937 - val_loss: 0.4685 - val_accuracy: 0.8125
Epoch 6/100
80/80 [==============================] - 44s 545ms/step - loss: 0.4971 - accuracy: 0.8094 - val_loss: 0.4927 - val_accuracy: 0.8250
Epoch 7/100
80/80 [==============================] - 44s 546ms/step - loss:

In [0]:
model.save_weights("/content/drive/My Drive/DeepFakeFiles/Checkpoints/")

In [0]:
EPOCHS = 50
model = tf.keras.models.load_model("/content/drive/My Drive/DeepFakeFiles/SecondModel.h5")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','val_acc'])
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(verbose=1,monitor='loss',patience=5),tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/DeepFakeFiles/SecondModel.h5',verbose=1,monitor='loss',save_best_only=True,save_freq='epoch',save_weights_only=True),tf.keras.callbacks.TensorBoard()]
print("Starting training")
model.fit(dataset,epochs=EPOCHS,callbacks=callbacks,steps_per_epoch=100,validation_data=valid,validation_steps=20)

In [0]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir '/content/logs'

TESTING PHASE

In [0]:
test_dataset = tf.data.TFRecordDataset("/content/drive/My Drive/DeepFakeFiles/test.tfrecords")

features_test = {'test/image': tf.io.FixedLenFeature([], tf.string),
            'test/label': tf.io.FixedLenFeature([], tf.int64),
           'test/audio': tf.io.FixedLenFeature([], tf.string),}

def _parse_function_test(example_proto):
  features = tf.io.parse_single_example(example_proto, features_test)
  audio_raw = tf.io.decode_raw(features['test/audio'],tf.float32)
  audio = tf.reshape(audio_raw,shape=(105000,1))

  image_raw = tf.io.decode_raw(features['test/image'],tf.float32)
  image = tf.reshape(image_raw,[5,500,500,3])
  image = tf.image.rgb_to_grayscale(image)
  image = tf.image.resize(image,(350,350))
  labels =features['test/label']
  labels = tf.one_hot(labels,depth=1) 
  return (image,audio),labels

In [0]:
test_dataset = test_dataset.map(_parse_function_test)
print(test_dataset)

<MapDataset shapes: (((5, 350, 350, 1), (105000, 1)), (1,)), types: ((tf.float32, tf.float32), tf.float32)>


In [0]:
test = test_dataset.batch(BATCH_SIZE).repeat()
test

<RepeatDataset shapes: (((None, 5, 350, 350, 1), (None, 105000, 1)), (None, 1)), types: ((tf.float32, tf.float32), tf.float32)>

In [0]:

model = tf.keras.models.load_model("/content/drive/My Drive/DeepFakeFiles/DeepFakeModel.h5")
pred = model.predict(test,steps=100,verbose=1)
print(pred)

100/100 [==============================] - 131s 1s/step
[[4.91826739e-08]
 [6.41587451e-02]
 [1.08890049e-01]
 [1.36651397e-01]
 [6.82397140e-03]
 [4.99885827e-01]
 [9.60812271e-01]
 [2.19488011e-05]
 [9.99962926e-01]
 [8.96965340e-02]
 [4.59584929e-02]
 [7.35937238e-06]
 [5.38711250e-02]
 [1.00000000e+00]
 [3.39367129e-02]
 [1.05553642e-02]
 [1.25744293e-04]
 [1.57567528e-10]
 [1.86705403e-03]
 [2.27522268e-09]
 [2.86584691e-06]
 [1.00000000e+00]
 [2.83398037e-07]
 [1.71117187e-01]
 [9.05862635e-06]
 [9.16126091e-03]
 [9.89413202e-01]
 [1.76853556e-02]
 [5.82880937e-02]
 [1.44009873e-01]
 [6.04730174e-02]
 [1.32031962e-01]
 [4.99885827e-01]
 [4.99885827e-01]
 [1.25744293e-04]
 [2.09695369e-01]
 [6.54853821e-01]
 [6.84426874e-02]
 [2.66822837e-02]
 [3.64856482e-01]
 [2.94143980e-08]
 [1.73023082e-02]
 [2.54347716e-02]
 [5.64808249e-01]
 [1.08890049e-01]
 [1.15575112e-01]
 [5.71611404e-01]
 [3.56033146e-01]
 [5.58459287e-05]
 [3.74714799e-12]
 [1.04572615e-02]
 [1.29941687e-01]
 [3.3777

In [0]:
np.save("/content/drive/My Drive/DeepFakeFiles/results.npy",pred)